# DATA COLLECTION AND PREP

# 0.0 IMPORTS

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import pandas as pd
from datetime import datetime
import os



import warnings
warnings.filterwarnings("ignore")

## 0.1 Load Data

In [2]:
df_metadados=pd.DataFrame()

for arquivo in os.listdir('/home/tc0019/DS/car_price_prediction/data/raw/year/'):
    print(arquivo)
    df_metadados=pd.concat([df_metadados, pd.read_csv('/home/tc0019/DS/car_price_prediction/data/raw/year/'+arquivo)])
df_metadados.reset_index(drop=True,inplace=True)


ce_metadados.csv
go_metadados.csv
pb_metadados.csv
sp_metadados.csv
pr_metadados.csv
df_metadados.csv
rn_metadados.csv
to_metadados.csv
mt_metadados.csv
mg_metadados.csv
ba_metadados.csv
rj_metadados.csv
pe_metadados.csv
ac_metadados.csv
rs_metadados.csv
ro_metadados.csv
pi_metadados.csv
am_metadados.csv
ms_metadados.csv
rr_metadados.csv
se_metadados.csv
ma_metadados.csv
pa_metadados.csv
ap_metadados.csv
al_metadados.csv
sc_metadados.csv
es_metadados.csv


Checking dataframe

In [3]:
df_metadados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122890 entries, 0 to 122889
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     122890 non-null  int64  
 1   TITULO                 122890 non-null  object 
 2   INFOS                  122890 non-null  object 
 3   VALOR                  117147 non-null  object 
 4   LOCAL                  122890 non-null  object 
 5   DATA_ANUNCIO           122890 non-null  object 
 6   LINK                   122890 non-null  object 
 7   DATA_COLETA_METADADOS  122890 non-null  object 
 8   DATA_COLETA_COMPLETA   0 non-null       float64
 9   UF                     89371 non-null   object 
 10  Ano                    104947 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 10.3+ MB


In [4]:
# drop duplicated ID's
df_metadados_limpos=df_metadados.drop_duplicates(subset=['ID']).reset_index(drop=True)
df_metadados_limpos.shape

(101000, 11)

# 1.0 Data Description

## 1.1 Check NA

In [5]:
print("==== INDICE DE NULOS NOS ATRIBUTOS ====")
dados = df_metadados_limpos.copy()
(dados.isnull().sum()/len(dados)*100).sort_values(ascending = False)

==== INDICE DE NULOS NOS ATRIBUTOS ====


DATA_COLETA_COMPLETA     100.000000
Ano                       15.450495
UF                        12.785149
VALOR                      3.378218
ID                         0.000000
TITULO                     0.000000
INFOS                      0.000000
LOCAL                      0.000000
DATA_ANUNCIO               0.000000
LINK                       0.000000
DATA_COLETA_METADADOS      0.000000
dtype: float64

In [6]:
# Drop NAN values
df_metadados_limpos.dropna(subset=['VALOR'], inplace=True)

# Drop Columns with many NAN values
df_metadados_limpos.drop(columns=['DATA_COLETA_COMPLETA'], inplace=True)

df_metadados_limpos.reset_index(inplace=True,drop=True)

## 1.2 PREPARAÇÃO/TRATAMENTO NOS DADOS

- Com base no que foi analisado inicialmente nos dados, vamos realizar os seguintes procedimentos para tratar os dados
    - Converter todos os atributos para minúsculo
    - INFOS: dividir em km, cambio e combustivel em novas colunas. Retirar ponto do KM e converter para float
    - VALOR: limpar R$, retirar ponto e converter para float
    - LOCAL: excluir coluna (já temos a UF)
    - DATA_ANUNCIO: excluir coluna
    - LINK: excluir coluna
    - DATA_COLETA: excluir coluna.
    - UF: corrigir linhas NULAS utilizando a coluna LINK para recuperar a UF

In [7]:
df_metadados_limpos = df_metadados_limpos.applymap(lambda x: x.lower() if type(x) == str else x)
df_metadados_limpos.head()

,ID,TITULO,INFOS,VALOR,LOCAL,DATA_ANUNCIO,LINK,DATA_COLETA_METADADOS,UF,Ano
0,1028211161,silverado 99.28 mil,0 km | câmbio: manual | diesel |,r$ 28.000,"maracanaú, pajuçara - ddd 85","hoje, 21:09",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,NaN
1,1026520834,nissan kicks sl cvt 2019,32.124 km | câmbio: automático | flex |,r$ 103.900,"fortaleza, tauape - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2019.0
2,1016380244,hrv 2016 lx 1.8 flex!,85.000 km | câmbio: automático | flex |,r$ 86.900,"fortaleza, salinas - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2016.0
3,964674083,hilux sw4 srx 2019 diesel.,62.000 km | câmbio: automático | diesel |,r$ 305.900,"fortaleza, salinas - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2019.0
4,999438981,mitsubishi pajero dakar hpe 3.2 4x4 t.i dies 5...,61.605 km | câmbio: automático | diesel |,r$ 169.900,"fortaleza, eng luciano cavalcante - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2015.0


INFOS

In [8]:
df_metadados_limpos.INFOS[0]

'0 km\xa0\xa0|\xa0\xa0câmbio: manual\xa0\xa0|\xa0\xa0diesel\xa0\xa0|\xa0\xa0'

In [9]:
## cleaning unwanted characters
infos = df_metadados_limpos.INFOS[:].str.replace("km|câmbio|:|\xa0|\.|\s+","",regex=True)

In [10]:
infos=infos.str.replace('\|$', "",regex=True)
infos=infos.str.split('|')
infos


0                [0, manual, diesel]
1          [32124, automático, flex]
2          [85000, automático, flex]
3        [62000, automático, diesel]
4        [61605, automático, diesel]
                    ...             
97583         [145000, manual, flex]
97584          [65900, manual, flex]
97585              [0, manual, flex]
97586      [64052, automático, flex]
97587    [47934, automático, diesel]
Name: INFOS, Length: 97588, dtype: object

In [11]:
lista_infos=[i for i in infos]
lista_infos[:10]

[['0', 'manual', 'diesel'],
 ['32124', 'automático', 'flex'],
 ['85000', 'automático', 'flex'],
 ['62000', 'automático', 'diesel'],
 ['61605', 'automático', 'diesel'],
 ['50979', 'automático', 'flex'],
 ['59000', 'manual', 'flex'],
 ['95700', 'manual', 'flex'],
 ['17000', 'automático', 'gasolina'],
 ['36000', 'manual', 'flex']]

In [12]:
infos_veiculo = pd.DataFrame(lista_infos, columns= ['KM', 'CAMBIO', 'COMBUSTIVEL'])
df_metadados_limpos_2 = pd.concat([df_metadados_limpos, infos_veiculo], axis=1)
df_metadados_limpos_2.to_csv('/home/tc0019/DS/car_price_prediction/data/raw/metadados_limpos_2.csv')
df_metadados_limpos_2.head()

,ID,TITULO,INFOS,VALOR,LOCAL,DATA_ANUNCIO,LINK,DATA_COLETA_METADADOS,UF,Ano,KM,CAMBIO,COMBUSTIVEL
0,1028211161,silverado 99.28 mil,0 km | câmbio: manual | diesel |,r$ 28.000,"maracanaú, pajuçara - ddd 85","hoje, 21:09",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,NaN,0,manual,diesel
1,1026520834,nissan kicks sl cvt 2019,32.124 km | câmbio: automático | flex |,r$ 103.900,"fortaleza, tauape - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2019.0,32124,automático,flex
2,1016380244,hrv 2016 lx 1.8 flex!,85.000 km | câmbio: automático | flex |,r$ 86.900,"fortaleza, salinas - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2016.0,85000,automático,flex
3,964674083,hilux sw4 srx 2019 diesel.,62.000 km | câmbio: automático | diesel |,r$ 305.900,"fortaleza, salinas - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2019.0,62000,automático,diesel
4,999438981,mitsubishi pajero dakar hpe 3.2 4x4 t.i dies 5...,61.605 km | câmbio: automático | diesel |,r$ 169.900,"fortaleza, eng luciano cavalcante - ddd 85","hoje, 21:08",https://ce.olx.com.br/fortaleza-e-regiao/autos...,2022-05-04,ce,2015.0,61605,automático,diesel


VALOR

In [13]:
df_metadados_limpos_2.loc[:, 'VALOR_LIMPO'] = df_metadados_limpos_2.VALOR.str.replace('r\$|\.|\s+', '',regex=True)
df_metadados_limpos_2.sample(5)

,ID,TITULO,INFOS,VALOR,LOCAL,DATA_ANUNCIO,LINK,DATA_COLETA_METADADOS,UF,Ano,KM,CAMBIO,COMBUSTIVEL,VALOR_LIMPO
87008,1023809599,vende uma praca de taxi na barra santo antonio,0 km | câmbio: manual | flex |,r$ 15.000,barra de santo antônio - ddd 82,"23 abr, 08:50",https://al.olx.com.br/alagoas/autos-e-pecas/ca...,2022-05-03,NaN,2015,0,manual,flex,15000
39184,1016882239,jeep renegade longitude,62.000 km | câmbio: semi-automático | flex...,r$ 97.000,"salvador, armação - ddd 71","hoje, 17:25",https://ba.olx.com.br/grande-salvador/autos-e-...,2022-05-03,ba,2020.0,62000,semi-automático,flex,97000
31539,979717427,etios 2019/2020 1.5 x plus sedan 16v flex 4p a...,55.404 km | câmbio: automático | flex |,r$ 69.790,"cuiabá, poção - ddd 65","22 abr, 15:12",https://mt.olx.com.br/regiao-de-cuiaba/autos-e...,2022-05-04,mt,2020,55404,automático,flex,69790
39064,1005276498,strada freedon cd 1.4 2020 3 portas apenas 40 ...,40.000 km | câmbio: manual | flex |,r$ 83.999,"salvador, imbuí - ddd 71","hoje, 18:22",https://ba.olx.com.br/grande-salvador/autos-e-...,2022-05-03,ba,2020.0,40000,manual,flex,83999
78205,1021404405,ford ka 2003,0 km | câmbio: manual | gasolina |,r$ 5.000,"são luís, turu - ddd 98","17 abr, 10:23",https://ma.olx.com.br/regiao-de-sao-luis/autos...,2022-05-04,ma,2003,0,manual,gasolina,5000


UF

In [14]:
(df_metadados_limpos_2.UF.nunique(),df_metadados_limpos_2.UF.unique())

(21,
 array(['ce', 'go', 'pb', 'sp', 'pr', 'df', nan, 'to', 'mt', 'mg', 'ba',
        'pe', 'rs', 'ro', 'pi', 'ms', 'rr', 'se', 'ma', 'pa', 'sc', 'es'],
       dtype=object))

In [15]:
# getting uf from link column
df_metadados_limpos_2.LINK[:5].apply(lambda x: x[8:10])

0    ce
1    ce
2    ce
3    ce
4    ce
Name: LINK, dtype: object

In [16]:
# insert column value only with none value
ids_uf_none=df_metadados_limpos_2.query('UF.isnull()', engine='python')['ID']
ids_uf_none=ids_uf_none.dropna()

In [17]:
# replacing NONE with UF
df_metadados_limpos_2.loc[df_metadados_limpos_2.ID.isin(ids_uf_none),'UF'] = df_metadados_limpos_2.query(f"ID in {ids_uf_none.tolist()}").LINK.apply(lambda x: x[8:10])

In [18]:
print("==== INDICE DE NULOS NOS ATRIBUTOS ====")
dados = df_metadados_limpos_2.copy()
(dados.isnull().sum()/len(dados)*100).sort_values(ascending=False)

==== INDICE DE NULOS NOS ATRIBUTOS ====


Ano                      15.546993
COMBUSTIVEL               4.080420
CAMBIO                    1.937738
ID                        0.000000
TITULO                    0.000000
INFOS                     0.000000
VALOR                     0.000000
LOCAL                     0.000000
DATA_ANUNCIO              0.000000
LINK                      0.000000
DATA_COLETA_METADADOS     0.000000
UF                        0.000000
KM                        0.000000
VALOR_LIMPO               0.000000
dtype: float64

Validation and adjustments
- Just checking some transformations done.

In [19]:
df_metadados_limpos_2.CAMBIO.unique()

array(['manual', 'automático', 'semi-automático', None, 'diesel', 'flex',
       'gasolina', 'híbrido', 'álcool', 'gásnatural'], dtype=object)

Shifting fuel filled in a wrong way in cambio to fuel column

In [20]:
fuel=[ 'diesel', 'flex','gasolina', 'híbrido', 'álcool', 'gásnatural']
df_valida_cambio=df_metadados_limpos_2.query(f'CAMBIO.isin({fuel})', engine='python')[['KM', 'CAMBIO', 'COMBUSTIVEL']]
df_valida_cambio 

,KM,CAMBIO,COMBUSTIVEL
921,30000,diesel,None
1064,11000,diesel,None
1341,100000,diesel,None
1379,134000,flex,None
1816,10500,gasolina,None
...,...,...,...
95759,89000,gasolina,None
96126,8100,flex,None
96160,68000,diesel,None
97010,20000,gasolina,None


In [21]:
df_metadados_limpos_3=df_metadados_limpos_2.copy()
ids_errados=df_metadados_limpos_3.query(f'CAMBIO.isin({fuel})', engine='python')['ID']
ids_errados

921      1027981470
1064     1025923353
1341     1025432833
1379     1027778125
1816     1017013669
            ...    
95759    1027617365
96126    1027518603
96160    1025967204
97010    1027188328
97580    1026984719
Name: ID, Length: 779, dtype: int64

In [22]:
df_metadados_limpos_3.loc[df_metadados_limpos_3.ID.isin(ids_errados),'COMBUSTIVEL'] = df_metadados_limpos_3.query(f"ID in {ids_errados.tolist()}")['CAMBIO']
df_metadados_limpos_3.query(f'ID in{ids_errados.tolist()}')[['CAMBIO','COMBUSTIVEL']]

,CAMBIO,COMBUSTIVEL
921,diesel,diesel
1064,diesel,diesel
1341,diesel,diesel
1379,flex,flex
1816,gasolina,gasolina
...,...,...
95759,gasolina,gasolina
96126,flex,flex
96160,diesel,diesel
97010,gasolina,gasolina


Deleting wrong values in the cambio column

In [23]:
df_metadados_limpos_3.loc[df_metadados_limpos_3.ID.isin(ids_errados),'CAMBIO'] = None
df_metadados_limpos_3.query(f"ID in {ids_errados.tolist()}")[['CAMBIO','COMBUSTIVEL']]

,CAMBIO,COMBUSTIVEL
921,None,diesel
1064,None,diesel
1341,None,diesel
1379,None,flex
1816,None,gasolina
...,...,...
95759,None,gasolina
96126,None,flex
96160,None,diesel
97010,None,gasolina


In [24]:
# check unique values of each column
(df_metadados_limpos_3.CAMBIO.unique(), df_metadados_limpos_3.COMBUSTIVEL.unique())

(array(['manual', 'automático', 'semi-automático', None], dtype=object),
 array(['diesel', 'flex', 'gasolina', 'gásnatural', None, 'álcool',
        'híbrido', 'elétrico'], dtype=object))

ANO

In [25]:
df_metadados_limpos_3.Ano.unique()

array([nan, 2019.0, 2016.0, 2015.0, 2008.0, 2013.0, 2017.0, 2018.0,
       2020.0, 2021.0, 2022.0, 2014.0, 2012.0, 2023.0, 2009.0, 2010.0,
       2011.0, 2003.0, 1991.0, 2005.0, 2002.0, 1999.0, 2006.0, 1997.0,
       2007.0, 1988.0, 1996.0, 1995.0, 2001.0, 1990.0, 1970.0, 1975.0,
       1994.0, 2004.0, 2000.0, 1993.0, 1983.0, 1992.0, 1986.0, 1998.0,
       1960.0, 1989.0, 1985.0, 1981.0, 1980.0, 1965.0, 1987.0, 1974.0,
       1982.0, 1955.0, 1984.0, '2019', '2015', '2010', '2003', '2013',
       '2020', '2011', '2018', '2005', '2017', '2008', '1980', '2009',
       '1997', '2014', '2021', '2022', '2012', '2006', '2000', '2016',
       '1995', '1998', '1996', '2007', '1988', '2002', '2023', '1960',
       '2004', '1999', '2001', '1994', '1993', '1975', '1989', '1991',
       '1970', '1986', '1985', '1990', '1992', '1965', '1987', '1981',
       '1984', '1950 ou anterior', '1982', 1978.0, '1971', '1983', '1955',
       '1973', '1976', '1978', '1974', 1979.0], dtype=object)

In [26]:
df_metadados_limpos_3.Ano = df_metadados_limpos_3.Ano.astype(str).str.replace(' ou anterior', "")
df_metadados_limpos_3[['KM','VALOR_LIMPO','Ano']] = df_metadados_limpos_3[['KM','VALOR_LIMPO','Ano']].apply(lambda x: x.astype(float))

In [27]:
## DELETING LINES WITHOUT VALUES
# this type of procedure should not be carried out the first time, but for didactic reasons this procedure will be applied.

df_metadados_limpos_3.dropna(inplace=True)
len(df_metadados_limpos_3)
## SELECIONANDO APENAS AS COLUNAS DE INTERESSE
df_metadados_limpos_3[['ID', 'TITULO','UF', 'Ano', 'KM', 'CAMBIO', 'COMBUSTIVEL','VALOR_LIMPO']].to_csv("/home/tc0019/DS/car_price_prediction/data/raw/df_metadados_limpos_3.csv",index=False)

In [28]:
print("==== NULL INDEX IN ATTRIBUTES ====")
dados = df_metadados_limpos_3.copy()
(dados.isnull().sum()/len(dados)*100).sort_values(ascending=False)


==== NULL INDEX IN ATTRIBUTES ====


ID                       0.0
TITULO                   0.0
INFOS                    0.0
VALOR                    0.0
LOCAL                    0.0
DATA_ANUNCIO             0.0
LINK                     0.0
DATA_COLETA_METADADOS    0.0
UF                       0.0
Ano                      0.0
KM                       0.0
CAMBIO                   0.0
COMBUSTIVEL              0.0
VALOR_LIMPO              0.0
dtype: float64